In [ ]:
#VIN3 model
import numpy as np
import parameters as p
import runModel
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm #for progress meter
import math
import datetime

df = pd.read_csv('Data.csv')#change name of file if importing a different dataset
#open and read file
df = df.dropna(how='any')
gen = df.loc[(df.Genotype == 'BRON') & (df.Experiment == 1)]#change Genotype name if using different genotype #experiment signifier added because the original dataset contained data from various experiments
#creates data frame for the TEST_VARN genotype from experiment 1
gen = gen.drop_duplicates(subset=['Timepoint_Cold','Temperature','Timepoint'])
#removes any duplicate rows
gen = gen.sort_values("Timepoint")
#presents data in data frame by ascending Timepoint
timetemp = (np.array([gen['Timepoint_Cold'],gen['Temperature'],gen['Timepoint']]))
#makes the data frame into an array containing the [Timepoint_Cold], [Temperature], [Timepoint] data

daysCold_all= timetemp[0,1:]
#creates array of all cold treatment lengths (days kept at cold temperature) except from the first one (initial conditions)
temp_all= timetemp[1,1:]
#creates array of all temperatures at times of sampling except from the first one (initial conditions)
days_all= timetemp[2,1:]
#creates array of all times of sampling (days since start of cold treatment) except from the first one (initial conditions)
timeTC = set() #keeps unique values, not ordered
#creates empty set timeTC

for i in range(len(daysCold_all)):
        days=days_all[(daysCold_all==daysCold_all[i]) & (temp_all==temp_all[i])] #adds all days where both cold duration and temperature match to array
        temp=temp_all[i]
        daysCold=daysCold_all[i]
        timeTC.add((tuple(days), temp, daysCold))#adds the loop values to the set
        #populates timeTC set with all combinations of days_all, temp_all and daysCold_all except from the initial conditions


#sets up initial VARN values and normalises them to initial Col FRI values
VIN3_all = []
#sets up empty array
COLFRIinit = df.loc[(df.Genotype == 'CF') & (df.Timepoint == 0) & (df.Timepoint_Cold == 0) & (df.Experiment == 1)] #The genotype name used should import initial Col FRI data from the used dataset
#imports initial TEST_CF data (before the cold treatment) for experiment 1
CFinit = np.array(COLFRIinit.VIN3)
#creates array of CFinit FLC values (initial FLC values before cold treatment)
df = df.loc[(df.Genotype == 'BRON') & (df.Experiment == 1)] #change Genotype name if using different genotype
#standardises VARN FLC values in data frame to mean initial Col FRI FLC value
VARNin=df.loc[(df.Genotype == 'BRON') & (df.Timepoint == 0) & (df.Timepoint_Cold == 0) & (df.Experiment == 1)] #change Genotype name if using different genotype
#creates data frame for initial VARN FLC values (when it hasn't beeen put in the cold)
VARNin_vin3=np.array(VARNin.VIN3)
#creates array of initial VARN FLC values
VARNin_TT=np.array(VARNin.Timepoint)
#creates an array of initial VARN timepoints (should be 0)

dV = np.arange(6,18, 1) #original = [18]
dB = np.logspace(-4, -1, num = 5) #original = [0.009]
A1 = np.arange(0.4,2.4, 0.4) #original = [0.75]
C1 = np.logspace(-1.52, -1.38, num=5) #original = [0.0315]
D1 = np.arange(1.8, 2.8, 0.2) #original = [2.05]


SSElist = []
log10_SSElist = []
dVlist = []
dBlist = []
A1list = []
C1list = []
D1list = []
ValueErrors = []

for v in range (len(dV)):
        p.dV = dV[v]
        for b in range (len(dB)):
                p.dB = dB[b]
                for a in range (len(A1)):
                        p.A1 = A1[a]
                        for c in range (len(C1)):
                                p.C1 = C1[c]
                                for d in range (len(D1)):
                                        p.D1 = D1[d]

                                        try:
                                                SSE=0 #resetting SSE variable to 0 for the upcoming model run
                                                for entry in timeTC: #for test set, there is only 1 entry so only runs the loop once
                                                        days=np.array(entry[0])
                                                        #set days array to all the days(timepoints) present in the set
                                                        temp=entry[1]
                                                        #sets temp to temp defined in the set
                                                        daysCold=entry[2]
                                                        #sets cold treatment length to the cold treatment length defined in the set
                                                        tiTi=np.array([[-5,timetemp[0,0],timetemp[0,0]+0.01, daysCold, daysCold+0.01, 100],[22,timetemp[1,0],temp, temp, temp,22]])
                                                        #creates an array with values [time at which temperature occurs][temperature values corresponding to the times] 
                                                        #defines the times and temperatures for changeover between the cold and warm conditions, and also for initial and end points of the model(model starts at -5 days and ends at 100 days)
                                                        p.tiTi=tiTi
                                                        #sets parameter value to tiTi array
                                                        TT=np.array([tiTi[0][0],0,*days, tiTi[0][-1]])#[-1] just means last entry, * just means that it can have an arbritrary number of arguments
                                                        #makes an array of timepoints [-5(start of model),0,all days for which there are data,100(end of model)]
                                                        VIN3,FLC,TT,y=runModel.run_model(p.param,p.tiTi,TT)
                                                        #runs model for experimental data and initial and end timepoints
                                                        VIN3_all.append(VIN3)
                                                        #appends experimental FLC value generated from model to array to store it
                                                        VIN3ex_vin3 = []
                                                        VIN3ex_TT = []
                                                        for i in range(len(days)): #for every timepoint for which there is experimental data, compares the experimental values with the model generated values
                                                                gen = df.loc[(df.Genotype == 'BRON') & (df.Timepoint == days[i]) & (df.Temperature == temp) & (df.Timepoint_Cold == daysCold) & (df.Experiment == 1)] #change Genotype name if using different genotype
                                                                #creates data frame for day being run
                                                                VIN3_ex = np.array(gen.VIN3)
                                                                #creates array containing experimental FLC values
                                                                VIN3ex_vin3.extend(list(VIN3_ex))
                                                                #adds the elements from FLC_ex to the FLCex_flc list
                                                                sumsq = (VIN3_ex - VIN3[i+2])*(VIN3_ex - VIN3[i+2]) #squares the difference between the experimental data point for this specific timepoint and the data point predicted by the model
                                                                SSE = SSE + sumsq.sum()
                                                                #adds all the square differences together and adds this to the already established SSE value for the other days (timepoints) in this run
                                                                VIN3ex_TT.extend([days[i]]*len(VIN3_ex))
                                                                #adds timepoints corresponding to experimental FLC values in the FLCex_flc array
                                                                                                                                        
                                                                                                                                                                                        
                                                        p.tiTi=np.array([[-5,-0.01,0,daysCold,daysCold+0.01,100],[22,22,temp,temp,22,22]])
                                                        #resets tiTi array parameter to define temperatures of the changeover points and edges of the model 
                                                        TT=np.arange(p.tiTi[0][0], p.tiTi[0][-1], 0.25)
                                                        #creates an array of timepoints from -5 to 100 in 0.25 increments to plot the graph on
                                                        VIN3,FLC,TT,y=runModel.run_model(p.param,p.tiTi,TT)
                                                        #runs model for all the increments to produce the curved line graph that is shown
                                                        #Plot of FLC mRNA in the conditions specified as "entry" in "timeTC". For information on which conditions produced the plot: print(entry)     
                                                        plt.plot(TT, VIN3, VARNin_TT, VARNin_vin3, "r^", VIN3ex_TT, VIN3ex_vin3, 'r^')
                                                        plt.ylabel('VIN3 expression')
                                                        plt.xlabel('Days')
                                                        plt.show()
                                                        #Plot of proportion of cells in states of silencing
                                                        print(entry)                                             

                                                                                                                                        

                                                gen = df.loc[(df.Genotype == 'BRON') & (df.Timepoint == 0) & (df.Timepoint_Cold == 0) & (df.Experiment == 1)] #change Genotype name if using different genotype
                                                VIN3_exi = np.array(gen.VIN3)
                                                #adds initial FLC values (no cold treatment has occurred) for that experiment into an array

                                                sumsq = (VIN3_exi - VIN3[1])*(VIN3_exi - VIN3[1])
                                                        
                                                SSE = SSE + sumsq.sum()
                                                SSElist.append(SSE)
                                                dVlist.append(dV[v])
                                                dBlist.append(dB[b])
                                                A1list.append(A1[a])
                                                C1list.append(C1[c])
                                                D1list.append(D1[d])
                                                SSEarray = (SSElist, dVlist, dBlist, A1list, C1list, D1list)
                                                np.savetxt("VIN3_SSE_values.csv",SSEarray,delimiter=",")
                                                table = [["SSE", "dV", "dB", "A1", "C1", "D1"], [SSE, dV[v], dB[b], A1[a], C1[c], D1[d]]]
                                                for row in table:
                                                        print('| {:^20} | {:^20} |{:^20} |{:^20} |{:^20} |{:^20} |'.format(*row))
                                                        #makes a table showing the current parameters the model is using and their SSE #just a visual to allow the user to compare the graph produced for the parameters being run
                                        except ValueError:
                                                ValueErrors.append([dV[v], dB[b], A1[a], C1[c], D1[d]]) #Appends all conditions where an error occured into a list

                                
SSEmin=min(SSEarray[0])
#finds the smallest SSE value
print("Value Errors:")
print(ValueErrors)

#this loop finds the min SSE values and the "optimal fit" parameter values associated with it and then writes them to the minSSE_values.txt file
for i in range(len(SSEarray[0])):
        if SSEarray[0][i] == SSEmin:
                print('SSEmin:')
                table_SSEmin = [["SSE", "dV", "dB", "A1", "C1", "D1"], [SSEarray[0][i], SSEarray[1][i], SSEarray[2][i], SSEarray[3][i], SSEarray[4][i], SSEarray[5][i]]]
                for row2 in table_SSEmin:
                        print('| {:^20} | {:^20} |{:^20} |{:^20} |{:^20} |{:^20} |'.format(*row2))
                timearray = [str(datetime.datetime.now())]
                with open("VIN3_minSSE_values.txt", mode='a') as file:
                        for w in range (0,len(SSEarray)): 
                                file.write("\n")
                                file.write(str(table_SSEmin[0][w]))
                                file.write("\t")
                                file.write(str(SSEarray[w][i]))
                        now = datetime.datetime.now()
                        file.write(now.strftime('\n%H:%M:%S on %A, %B the %dth, %Y\n'))                                       

In [ ]:
#FLC MODEL
import numpy as np
import parameters as p
import runModel
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm #for progress meter
import math
import datetime

df = pd.read_csv('Data.csv')#change name of file if importing a different dataset
#open and read file
gen = df.loc[(df.Genotype == 'BRON') & (df.Experiment == 1)]#change Genotype name if using different genotype #experiment signifier added because the original dataset contained data from various experiments
#creates data frame for the TEST_VARN genotype from experiment 1
gen = gen.drop_duplicates(subset=['Timepoint_Cold','Temperature','Timepoint'])
#removes any duplicate rows
gen = gen.sort_values("Timepoint")
#presents data in data frame by ascending Timepoint
timetemp = (np.array([gen['Timepoint_Cold'],gen['Temperature'],gen['Timepoint']]))
#makes the data frame into an array containing the [Timepoint_Cold], [Temperature], [Timepoint] data

#define VIN3 parameters we're using
p.dV = 9.2
p.dB = 0.00316
p.A1 = 2
p.C1 = 0.0301995
p.D1 = 2.2

daysCold_all= timetemp[0,1:]
#creates array of all cold treatment lengths (days kept at cold temperature) except from the first one (initial conditions)
temp_all= timetemp[1,1:]
#creates array of all temperatures at times of sampling except from the first one (initial conditions)
days_all= timetemp[2,1:]
#creates array of all times of sampling (days since start of cold treatment) except from the first one (initial conditions)
timeTC = set() #keeps unique values, not ordered
#creates empty set timeTC

for i in range(len(daysCold_all)):
        days=days_all[(daysCold_all==daysCold_all[i]) & (temp_all==temp_all[i])] #adds all days where both cold duration and temperature match to array
        temp=temp_all[i]
        daysCold=daysCold_all[i]
        timeTC.add((tuple(days), temp, daysCold))#adds the loop values to the set
        #populates timeTC set with all combinations of days_all, temp_all and daysCold_all except from the initial conditions


#sets up initial VARN values and normalises them to initial Col FRI values
FLC_all = []
#sets up empty array
COLFRIinit = df.loc[(df.Genotype == 'CF') & (df.Timepoint == 0) & (df.Timepoint_Cold == 0) & (df.Experiment == 1)] #The genotype name used should import initial Col FRI data from the used dataset
#imports initial TEST_CF data (before the cold treatment) for experiment 1
CFinit = np.array(COLFRIinit.FLC)
#creates array of CFinit FLC values (initial FLC values before cold treatment)
df = df.loc[(df.Genotype == 'BRON') & (df.Experiment == 1)] #change Genotype name if using different genotype
df.FLC = df.FLC / np.mean(CFinit)
#standardises VARN FLC values in data frame to mean initial Col FRI FLC value
VARNin=df.loc[(df.Genotype == 'BRON') & (df.Timepoint == 0) & (df.Timepoint_Cold == 0) & (df.Experiment == 1)] #change Genotype name if using different genotype
#creates data frame for initial VARN FLC values (when it hasn't beeen put in the cold)
VARNin_flc=np.array(VARNin.FLC)
#creates array of initial VARN FLC values
VARNin_TT=np.array(VARNin.Timepoint)
#creates an array of initial VARN timepoints (should be 0)

rangearray = [0.3,1.5,0.05,0.05,0.002,0.02] #starting range, range is halfed every time that the program loops
optarray =   [0.3,5.5,0.05,0.05,0.002,0.02] #array of initial optimal values (halfway point for initial loop)

for bigloop in tqdm(range (0,10)): #can adjust range here and choose how many optimisation loops you want to run. Should "zone in" around the minimum value it finds to return more precise optimum parameters
        n0i = np.arange(optarray[0]-(rangearray[0]/(2**bigloop)), optarray[0]+2*(float(rangearray[0])/(2**bigloop)), rangearray[0]/(2**bigloop) )#Col FRI = 0.2 #Bro NIl = [0.178125]
        #n0i is the proportion of silenced cells initially
        pf = np.arange(optarray[1]-(rangearray[1]/(2**bigloop)), optarray[1]+2*(float(rangearray[1])/(2**bigloop)), rangearray[1]/(2**bigloop) )#Col FRI= 2.77, Var NIL = 7, Bro NIL = [6.765625]
        #pf is the degradation rate of the FLC mRNA amount #makes an array of values between 6 and 8 increasng by increments of 0.1
        pr2 = np.arange(optarray[2]-(rangearray[2]/(2**bigloop)), optarray[2]+2*(float(rangearray[2])/(2**bigloop)), rangearray[2]/(2**bigloop) )#Col FRI = 0, Var NIL = 0.001, Bro NIL = [0]
        #pr2 is the VIN3 dependent reversal rate (P to I) #makes an array of 10 values between 10^-3 and 10^0 spaced evenly on a log 10 scale
        pr1 = np.arange(optarray[3]-(rangearray[3]/(2**bigloop)), optarray[3]+2*(float(rangearray[3])/(2**bigloop)), rangearray[3]/(2**bigloop) )#Col FRI = 0.05, Var NIL = 0.028, Bro NIL = [0.0171875]
        #pr1 is the VIN3 independent reversal rate (I to H) #makes an array of 10 values between 10^-2 and 10^0 spaced evenly on a log 10 scale
        s2 = np.arange(optarray[4]-(rangearray[4]/(2**bigloop)), optarray[4]+2*(float(rangearray[4])/(2**bigloop)), rangearray[4]/(2**bigloop)) #Bro NIL = [0.005875]
        s1 = np.arange(optarray[5]-(rangearray[5]/(2**bigloop)), optarray[5]+2*(float(rangearray[5])/(2**bigloop)), rangearray[5]/(2**bigloop)) #Bro NIL = [0.016875]
        

        SSElist = []
        log10_SSElist = []
        n0ilist = []
        pflist = []
        pr2list = []
        pr1list = []
        s2list = []
        s1list = []
        ValueErrors = []


        #we now have initial conditions set up and all possible combinations of experimental data defined
        #now we test the model for all possible parameter combinations and see which one has the lowest SSE
        
        for s in range(len(s2)):
                p.param[1] = s2[s]
                for q in range (len(s1)):
                        p.param[0] = s1[q]
                        for n in range(len(n0i)):
                                p.param[7] = n0i[n]
                                for f in range(len(pf)):
                                        p.param[13] = pf[f]
                                        for h in range(len(pr2)):  
                                                p.param[5] = pr2[h]
                                                # for every value in variable array, set position in parameter array to current value for that variable (cycle through all the different variable combinations)
                                                for l in range(len(pr1)):
                                                        try:
                                                                p.param[4] = pr1[l]
                                                                #sets position in parameter array to current value for that variable       
                                                                SSE=0 #resetting SSE variable to 0 for the upcoming model run
                                                                for entry in timeTC: #for test set, there is only 1 entry so only runs the loop once
                                                                        days=np.array(entry[0])
                                                                        #set days array to all the days(timepoints) present in the set
                                                                        temp=entry[1]
                                                                        #sets temp to temp defined in the set
                                                                        daysCold=entry[2]
                                                                        #sets cold treatment length to the cold treatment length defined in the set
                                                                                

                                                                        tiTi=np.array([[-5,timetemp[0,0],timetemp[0,0]+0.01, daysCold, daysCold+0.01, 100],[22,timetemp[1,0],temp, temp, temp,22]])
                                                                        #creates an array with values [time at which temperature occurs][temperature values corresponding to the times] 
                                                                        #defines the times and temperatures for changeover between the cold and warm conditions, and also for initial and end points of the model(model starts at -5 days and ends at 100 days)
                                                                        p.tiTi=tiTi
                                                                        #sets parameter value to tiTi array
                                                                        TT=np.array([tiTi[0][0],0,*days, tiTi[0][-1]])#[-1] just means last entry, * just means that it can have an arbritrary number of arguments
                                                                        #makes an array of timepoints [-5(start of model),0,all days for which there are data,100(end of model)]
                                                                        VIN3,FLC,TT,y=runModel.run_model(p.param,p.tiTi,TT)
                                                                        #runs model for experimental data and initial and end timepoints
                                                                        FLC_all.append(FLC)
                                                                                #appends experimental FLC value generated from model to array to store it
                                                                        FLCex_flc = []
                                                                        FLCex_TT = []
                                                                        for i in range(len(days)): #for every timepoint for which there is experimental data, compares the experimental values with the model generated values
                                                                                gen = df.loc[(df.Genotype == 'BRON') & (df.Timepoint == days[i]) & (df.Temperature == temp) & (df.Timepoint_Cold == daysCold) & (df.Experiment == 1)] #change Genotype name if using different genotype
                                                                                #creates data frame for day being run
                                                                                FLC_ex = np.array(gen.FLC)
                                                                                #creates array containing experimental FLC values
                                                                                FLCex_flc.extend(list(FLC_ex))
                                                                                #adds the elements from FLC_ex to the FLCex_flc list, creates permanent copy of experimental data?
                                                                                sumsq = (FLC_ex - FLC[i+2])*(FLC_ex - FLC[i+2])
                                                                                #squares the difference between the experimental data point for this specific timepoint and the data point predicted by the model
                                                                                SSE = SSE + sumsq.sum()
                                                                                #adds all the square differences together and adds this to the already established SSE value for the other days (timepoints) in this run
                                                                                FLCex_TT.extend([days[i]]*len(FLC_ex))
                                                                                #adds timepoints corresponding to experimental FLC values in the FLCex_flc array
                                                                                
                                                                                                                                
                                                                        p.tiTi=np.array([[-5,-0.01,0,daysCold,daysCold+0.01,100],[22,22,temp,temp,22,22]])
                                                                        #resets tiTi array parameter to define temperatures of the changeover points and edges of the model 
                                                                        TT=np.arange(p.tiTi[0][0], p.tiTi[0][-1], 0.25)
                                                                        #creates an array of timepoints from -5 to 100 in 0.25 increments to plot the graph on
                                                                        VIN3,FLC,TT,y=runModel.run_model(p.param,p.tiTi,TT)
                                                                        #runs model for all the increments to produce the curved line graph that is shown

                                                                        #Plot of FLC mRNA in the conditions specified as "entry" in "timeTC". For information on which conditions produced the plot: print(entry)     
                                                                        plt.plot(TT, FLC, VARNin_TT, VARNin_flc, "r^", FLCex_TT, FLCex_flc, 'r^')
                                                                        plt.ylabel('FLC expression')
                                                                        plt.xlabel('Days')
                                                                        plt.show()
                                                                        #Plot of proportion of cells in states of silencing
                                                                        plt.plot(TT,y[:,3:8])
                                                                        # plots with values TT and values in the first column of y in positions 3-8
                                                                        plt.legend(['H','I','N','S','P'])
                                                                        plt.ylabel('Proportion of cells')
                                                                        plt.xlabel('Days')
                                                                        plt.show() 
                                                                        print(entry)                                             

                                                                                

                                                                gen = df.loc[(df.Genotype == 'BRON') & (df.Timepoint == 0) & (df.Timepoint_Cold == 0) & (df.Experiment == 1)] #change Genotype name if using different genotype
                                                                FLC_exi = np.array(gen.FLC)
                                                                #adds initial FLC values (no cold treatment has occurred) for that experiment into an array

                                                                sumsq = (FLC_exi - FLC[1])*(FLC_exi - FLC[1])
                                                                SSE = SSE + sumsq.sum()
                                                                SSElist.append(SSE)
                                                                #calculates the squared difference for the initial FLC value and adds this to the rest of SSE values
                                                                pflist.append(pf[f])
                                                                pr1list.append(pr1[l])
                                                                pr2list.append(pr2[h])
                                                                n0ilist.append(n0i[n])
                                                                s2list.append(s2[s])
                                                                s1list.append(s1[q])
                                                                SSEarray = (SSElist, n0ilist, pflist, pr2list, pr1list,s2list,s1list)
                                                                np.savetxt("SSE_values.csv",SSEarray,delimiter=",")
                                                                #saves a text document with the paramaters run and their corresponding SSE


                                                                log10_SSE = math.log10(SSE)
                                                                log10_SSElist.append(log10_SSE)
                                                                log10_SSEarray = (log10_SSElist, n0ilist, pflist, pr2list, pr1list,s2list,s1list)
                                                                df_log = pd.DataFrame(zip(log10_SSElist, n0ilist, pflist, pr2list, pr1list,s2list,s1list), columns=['logSSE','n0i','pf','pr2','pr1','s2','s1'])
                                                                df_log.to_csv('log10(SSE_values).csv', sep=',', index=False)
                                                                #converts SSE into log10SSE and saves the log10SSE values in a document

                                                                table = [["SSE", "n0i","pf","pr2","pr1","s2","s1"], [SSE, n0i[n], pf[f], pr2[h],pr1[l],s2[s],s1[q]]]
                                                                for row in table:
                                                                        print('| {:^20} | {:^20} | {:^20} | {:^20} |{:^20} |{:^20} |{:^20} |'.format(*row))
                                                                        #makes a table showing the current parameters the model is using and their SSE #just a visual to allow the user to compare the graph produced for the parameters being run
                                                        except ValueError:
                                                                ValueErrors.append([n0i[n], pf[f], pr2[h], pr1[l],s2[s],s1[q]]) #Appends all conditions where an error occured into a list
                                                                #reports any errors that occur
                                        
        SSEmin=min(SSEarray[0])
        #finds the smallest SSE value
        print("Value Errors:")
        print(ValueErrors)

        #this loop finds the min SSE values and the "optimal fit" parameter values associated with it and then writes them to the minSSE_values.txt file
        for i in range(len(SSEarray[0])):
                if SSEarray[0][i] == SSEmin:
                        print('SSEmin:')
                        table_SSEmin = [["SSE", "n0i","pf","pr2","pr1","s2","s1"], [SSEarray[0][i], SSEarray[1][i], SSEarray[2][i], SSEarray[3][i], SSEarray[4][i], SSEarray[5][i],SSEarray[6][i]]]
                        for row2 in table_SSEmin:
                                print('| {:^20} | {:^20} | {:^20} | {:^20} |{:^20} |{:^20} |{:^20} |'.format(*row2))
                        timearray = [str(datetime.datetime.now())]
                        with open("minSSE_values.txt", mode='a') as file:
                                for w in range (0,7): 
                                        file.write("\n")
                                        file.write(str(table_SSEmin[0][w]))
                                        file.write("\t")
                                        file.write(str(SSEarray[w][i]))
                                now = datetime.datetime.now()
                                file.write(now.strftime('\n%H:%M:%S on %A, %B the %dth, %Y\n'))
                        optarray = [SSEarray[1][i], SSEarray[2][i], SSEarray[3][i], SSEarray[4][i], SSEarray[5][i],SSEarray[6][i]]


In [ ]:
#gives you the minimum log10(SSE) in the csv file #can be used when the optimisation program is stopped before it is complete
import pandas as pd
import numpy as np

df = pd.read_csv('log10(SSE_values).csv')
min =df['logSSE'].min()
rows = df.shape[0]-1
for loop in range (1,rows):
    if df.loc[loop][0]==min:
        print(df.loc[[loop]])
        break

In [ ]:
#Code to create Heatmaps. Not used in this project as too many variables are being tested
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
df = pd.read_csv('log10(SSE_values).csv') #SSE values output from above script. Log10 values of SSE calculated in Excel. 
sse1 = df.loc[(df.n0i == 0) & (df.pr2 == 0.001)]
sse2 = df.loc[(df.n0i == 0) & (df.pr2 == 0.01)]
sse3 = df.loc[(df.n0i == 0) & (df.pr2 == 0.1)]
sse4 = df.loc[(df.n0i == 0) & (df.pr2 == 1)]
#creates data sets for different conditions
ssearray1 = np.array([sse1.pf, sse1.pr1])
ssearray2 = np.array([sse2.pf, sse2.pr1])
ssearray3 = np.array([sse3.pf, sse3.pr1])
ssearray4 = np.array([sse4.pf, sse4.pr1])
#creates arrays of pf and pr1 values for the above conditions
threeval1 = sse1[["logSSE","pf","pr1"]]
threeval2 = sse2[["logSSE","pf","pr1"]]
threeval3 = sse3[["logSSE","pf","pr1"]]
threeval4 = sse4[["logSSE","pf","pr1"]]
#create data frames containing only logSSE, pf and pr1 values
pivoted1 = threeval1.pivot("pf","pr1","logSSE")
pivoted2 = threeval2.pivot("pf","pr1","logSSE")
pivoted3 = threeval3.pivot("pf","pr1","logSSE")
pivoted4 = threeval4.pivot("pf","pr1","logSSE")
#data frames are reshaped to a different format
fig, ax = plt.subplots(figsize = (25,5), ncols=4)
sns.heatmap(pivoted1, vmin = 0.6, vmax = 1.5, cbar = False, ax=ax[0])
sns.heatmap(pivoted2, vmin = 0.6, vmax =1.5, cbar = False, ax=ax[1])
sns.heatmap(pivoted3, vmin = 0.6, vmax = 1.5, cbar = False, ax=ax[2])
sns.heatmap(pivoted4, vmin = 0.6, vmax =1.5, cbar = True, ax=ax[3])
#create heatmap with columns displaying different pivoted values

plt.show()